In [ ]:
from energy.maximum_energy_calculate import maximum_energy_calculate
import pandas as pd
plc = [40]
ts = maximum_energy_calculate(plc)
print(ts)
if isinstance(ts, pd.DataFrame):
    ts.to_csv('try.csv')

In [ ]:
from clock.get_current_time import get_current_time
from clock.time_count import time_count
stime = '2022-09-04 '+get_current_time()[1]
end_time = get_current_time()[0]
time_count(stime, end_time)

In [ ]:
from energy.maximum_energy_calculate import maximum_energy_calculate
from energy.median_energy_calculate import median_energy_calculate
from clock.get_current_time import get_current_time
import datetime
from settings import fmt1
start_time = '2022-09-16 14:00:00'
end_time = get_current_time()[0]
median_energy_calculate(start_time, end_time, [40])

In [ ]:
from mongodb.query_request import query_request
import datetime
from settings import fmt1, filter_for_energy_calculation
start_time = '2022-09-15 08:00:00'
end_time = '2022-09-15 08:05:00'
start_time_ts = datetime.datetime.strptime(start_time, fmt1)
end_time_ts = datetime.datetime.strptime(end_time, fmt1)
data = query_request.datalog_query(start_time_ts, end_time_ts, [48], filter_for_energy_calculation)
data

In [ ]:
from sched import scheduler
from clock.get_current_time import get_current_time
from energy.energy_trigger import energy_trigger
from apscheduler.schedulers.blocking import BlockingScheduler
start_time = get_current_time()[0]
def et_try():
    print('---Begin---')
    plc = [2]
    flag = energy_trigger(plc)
    print(get_current_time()[0], flag)
    print('---end---')
scheduler = BlockingScheduler()
scheduler.add_job(et_try, 'interval', seconds = 2)
scheduler.start()

In [ ]:
from sched import scheduler
from clock.get_current_time import get_current_time
from energy.energy_trigger import energy_trigger
from apscheduler.schedulers.blocking import BlockingScheduler
from plcid.query_all_plc import query_all_plc
from settings import plc_csv_file, plc_csv_path
import pandas as pd
start_time = get_current_time()[0]
def diagnose_trigger_service():
    print('---Begin---')
    plc_update = query_all_plc()
    df_plc = pd.read_csv(plc_csv_path+plc_csv_file)
    plc_set = df_plc['plc'].values
    for i in range(0, df_plc.shape[0]):
        plc = [int(plc_set[i])]
        print(plc)
        flag = energy_trigger(plc)
        print(get_current_time()[0], flag)
    print('---end---')
scheduler = BlockingScheduler()
scheduler.add_job(diagnose_trigger_service, 'interval', seconds = 15)
scheduler.start()

In [14]:
import pandas as pd
a = pd.DataFrame({'A':[1,2,3], 'B':[1,2,3]})
b = pd.DataFrame({'A':[4,5,6], 'B':[4,5,6]})
c = pd.concat([a,b], axis=0, ignore_index=True)

In [ ]:
from messager.dingtalk_chatbot import dingtalkchatrobot
import pandas as pd
from settings import plc_csv_file, plc_csv_path
#dingtalkchatrobot.energy_trigger_sender()
df_plc = pd.read_csv(plc_csv_path+plc_csv_file)
plcset = df_plc['plc'].values
minus_set = []
zero_set = []
one_set = []
for plc in plcset:
    plc = [plc]
    energy_flag = dingtalkchatrobot.get_trigger(plc)
    if energy_flag == 0:
        zero_set.append(energy_flag)
    if energy_flag == -1:
        zero_set.append(energy_flag)
    if energy_flag == 1:
        zero_set.append(energy_flag)
print('0:', zero_set)
print('-1:', minus_set)
print('1:', one_set)

In [ ]:
from mongodb.query_request import query_request
from clock.get_current_time import get_current_time
import datetime
from settings import energy_trigger_sender_webhook, \
    energy_trigger_sender_secret, bot_query_dayrange, filter_for_energy_trigger, fmt1
now_time = get_current_time()[0]
now_time_ts = datetime.datetime.strptime(now_time, fmt1)
start_time_ts = now_time_ts - datetime.timedelta(days=bot_query_dayrange)
print(start_time_ts ,now_time_ts)
data = query_request.trigger_query(start_time_ts, now_time_ts, [2], \
    filter_for_energy_trigger)

In [ ]:
threshold = [1,1]
max = [2,2]
plc = [2]
text_report = 'PLC_id:'+str(plc[0])+', '+'[充电量标准]forward:'+str(threshold[0])+', [放电量标准]reverse:'+str(threshold[0])+',\n'\
                +'[24小时内最大充电量]forward:'+str(max[0])+', [24小时内最大放电量]reverse:'+str(max[1])+'.'
print(text_report)

In [ ]:
import pandas as pd
df = pd.DataFrame({'A':[-1,-2,-3]})
b = abs(df['A']).max()
c = df[abs(df['A'])>1].index
col = df.columns.get_loc('A')
df.iloc[c.values, col] = 0
df

In [ ]:
from mongodb.query_request import query_request
from settings import plc_csv_file, plc_csv_path, filter_for_energy_threshold, fmt1
import pandas as pd
from clock.get_current_time import get_current_time
import datetime
df_plc_list = pd.read_csv(plc_csv_path+plc_csv_file)
plcset = df_plc_list['plc'].values
df_thres = pd.DataFrame()
for i in range(0, len(plcset)):
    plc = [int(plcset[i])]
    print(plc)
    start_time = str(df_plc_list[df_plc_list['plc']==plc[0]]['time'].values[0])
    start_time_ts = datetime.datetime.strptime(start_time, fmt1)
    now_time = get_current_time()
    now_time_ts = datetime.datetime.strptime(now_time[0], fmt1)
    data = query_request.threshold_query(start_time_ts, now_time_ts, plc, filter_for_energy_threshold)
    if len(data) > 0:
        thres_f = data[0]['median_max_forward']
        thres_r = data[0]['median_max_reverse']
        if thres_f >= 0 and thres_r >= 0:
            print(start_time_ts)
            thres_end_time_ts = start_time_ts + datetime.timedelta(days=7)
            print(thres_end_time_ts)
            to_append = pd.DataFrame({'plc': [plc[0]], '开始时间': [start_time_ts], '结束时间': [thres_end_time_ts], \
                                      '最大单次充电量': [thres_f], '最大单次放电量': [thres_r]})
            df_thres = df_thres.append(to_append)
df_thres = df_thres.sort_values(by = 'plc', ascending=True)
df_thres = df_thres.reset_index(drop=True)
df_thres = df_thres.loc[:, ~df_thres.columns.str.contains("^Unnamed")]
df_thres.to_csv('有效最大单次充放电量.csv')

In [ ]:
from data.dron import dron
data = dron.find_battery_to_check([40])

In [ ]:
data['Voltage'].mean()

In [ ]:
data['Voltage'].std()

In [ ]:
data

In [ ]:
(3.410-data['Voltage'].mean())/data['Voltage'].std()

In [ ]:
2.815975*data['Voltage'].std()+data['Voltage'].mean()

In [ ]:
from soc.main_entry import SOC
import time
from mongodb.query_request import query_request
import importlib
from clock.get_current_time import get_current_time
import datetime
import numpy as np

settings = importlib.import_module('settings')
now_time = get_current_time()[0]
now_time = '2022-09-26 23:00:00'
plc = [40]
now_time_ts = datetime.datetime.strptime(now_time, settings.fmt1)
start_time_ts = now_time_ts - datetime.timedelta(days=settings.day_interval)
data = query_request.datalog_query(start_time_ts, now_time_ts, plc, \
                                    settings.filter_for_data_collection)
for i in range(0, len(data)):
    theTime = (data[i]['time']).strftime('%Y-%m-%d %H:%M:%S')
    theVolt = np.int16(data[i]['data']['1组1号电池电压'])/1000
    theCurr = np.int16(data[i]['data']['PCS电池电流'])/10
    theTemp = 25
    
    if i > 20:
        theStatus = -1
    else:
        theStatus = 0
    print(theTime, theVolt, theCurr, theTemp, theStatus)
    soc = SOC(40, theTime, theVolt, theCurr, theTemp, theStatus)
    print(soc.predict_soc)

In [6]:
from mysql.mysql_model import *
mysql_session = get_session()
thres_parameter = mysql_session.query(energy_threshold).filter(energy_threshold.plc_id==40).all()
thres_parameter[0].median_max_forward

215.6

In [6]:
import datetime
try1 = diagnose_trigger(plc_id=2, time = datetime.datetime.now(), energy_flag = 0)
mysql_session.add(try1)
mysql_session.commit()

In [22]:
import pandas as pd
pd.to_datetime(data.time)
mysql_session.query(diagnose_trigger).filter(diagnose_trigger.plc_id==1).update({'time':pd.to_datetime(datetime.datetime.now())})
mysql_session.commit()

In [20]:
mysql_session.close()

In [11]:
query[0].time

datetime.datetime(2022, 9, 30, 18, 3, 18)

In [17]:
# add/update data:
import sqlalchemy
import datetime
from pandas import to_datetime
from mysql.mysql_model import *
mysql_session = get_session()
query = mysql_session.query(diagnose_trigger).filter(diagnose_trigger.plc_id==3).all()
if len(query) == 1:
    mysql_session.query(diagnose_trigger).filter(diagnose_trigger.plc_id==3).update({'time':to_datetime(datetime.datetime.now()), 'energy_flag':0})
    mysql_session.commit()
else:
    if len(query) < 1:
        add = diagnose_trigger(plc_id=3, time=to_datetime(datetime.datetime.now()), energy_flag=1)
        mysql_session.add(add)
        mysql_session.commit()
    else:
        print('PLC_ID ' + str(3) + 'has more than one segment of data.')



In [11]:
a = 'A'
b = ['A', 'B']
(a in b)

'a' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
from soc.getParamESC import getParamESC
import scipy.io as scio
import numpy as np
matfile = './soc/280Ahmodel.mat'
model = scio.loadmat(matfile)['model']
#getParamESC('A', 25, model)
len(model['temps'][0][0][0])
param = 'dOCVrel'
model[param][0][0].shape[0]

1

In [11]:
a = 1
a = [a]
a = [a]


In [12]:
import numpy as np
np.array(a).reshape(1, )

array([1])

In [40]:
np.array([[1,2,3]])[0, :]

array([1, 2, 3])

In [48]:
from scipy.interpolate import make_interp_spline
x = np.array([1,2,3,4])
y = np.array([2,4,6,5])
make_interp_spline(x, y)(1)

array(2.)

In [1]:
from soc.getParamESC import getParamESC
import scipy.io as scio
matfile = './soc/280Ahmodel.mat'
model = scio.loadmat(matfile)['model']
getParamESC('dOCVrel', 22, model)

array([], dtype=float64)